ANN Final Project: Word Embedding

In [1]:
import numpy
%matplotlib inline

import numpy as np
import tensorflow
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet

import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = nltk.corpus.stopwords.words('english')
stop_words.append('cnn')
lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

import random
import sys
import io
import os
import datetime
from bs4 import BeautifulSoup
import requests as rq

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras



[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\edwin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edwin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edwin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\edwin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import keras
import pandas as pd
import re
import tensorflow.keras
'''
data = pd.read_csv('CNN_Articels_clean.csv')
text = pd.DataFrame(data, columns=['Article text'])
text_np = pd.DataFrame.to_numpy(text)
'''
lyrics_file = open('beatles.txt', 'r')
text = lyrics_file.read()
lyrics_file.close()

def tokenize(text):
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    return pattern.findall(text.lower())

def preprocess(tokens):
    tokens_processed = []
    for words in tokens:
        if words in stop_words:
            continue
        if len(words) <= 2:
            continue

        tokens_processed.append(words)

    return tokens_processed

def mapping(tokens):
    word_to_id = {}
    id_to_word = {}

    for i, token in enumerate(set(tokens)):
        word_to_id[token] = i
        id_to_word[i] = token

    print('mapping done')

    return word_to_id, id_to_word


def generate_training_data(tokens, word_to_id, window):
    X = []
    y = []
    n_tokens = len(tokens)

    for i in range(n_tokens):
        idx = concat(
            range(max(0, i - window), i),
            range(i, min(n_tokens, i + window + 1))
        )
        i_id = word_to_id[tokens[i]]
        for j in idx:
            if i == j:
                continue
            X.append(one_hot_encode(i_id, len(word_to_id)))
            y.append(one_hot_encode(word_to_id[tokens[j]], len(word_to_id)))

        if i%1000 == 0:
            print('generated'+ str(i) +' tokens')

    print('training data generated')
    return np.array(X), np.array(y)



def concat(*iterables):
    for iterable in iterables:
        yield from iterable


def one_hot_encode(id, vocab_size):
    res = [0] * vocab_size
    res[id] = 1
    return res

tokens = preprocess(tokenize(text))

#tokens = preprocess(tokenize(str(text_np[1])))

'''
for i in range(40):
    tokens += preprocess(tokenize(str(text_np[i+2])))
'''
print(np.shape(tokens))

"""
tokens = list(map(lemmatizer.lemmatize, tokens))
for words in tokens:
    if(words in stop_words):
        tokens.remove(words)
    elif(len(words) < 3):
        tokens.remove(words)

print(tokens)



#tokens = lemmatizer.lemmatize(tokens)

for i in range(15):
    temp_tokens = tokenize(str(text_np[i+2]))
    temp_tokens = list(map(lemmatizer.lemmatize, temp_tokens))

    for word in temp_tokens:
        if(word in stop_words):
            temp_tokens.remove(word)
        elif(len(word) < 3):
            temp_tokens.remove(word)
    tokens += temp_tokens

    print('tokenization #' + str(i))
"""
word_to_id, id_to_word = mapping(tokens)
print(word_to_id)


X, y = generate_training_data(tokens, word_to_id, 3)



#print(np.shape(X))
#print(np.shape(y))

print(X.shape)

(7260,)
mapping done
{'get': 0, 'times': 1, 'number': 2, 'stinking': 3, 'limitless': 4, 'kind': 5, 'cold': 6, 'flew': 7, 'leads': 8, 'air': 9, 'dawn': 10, 'memories': 11, 'dreams': 12, 'stick': 13, "dog's": 14, 'jojo': 15, 'suburban': 16, 'restless': 17, 'grade': 18, 'six': 19, 'rocking': 20, 'cuesto': 21, 'coca': 22, 'proceeded': 23, 'unpleasant': 24, 'near': 25, 'helloa': 26, 'story': 27, 'gas': 28, 'first': 29, 'south': 30, 'tune': 31, 'keeps': 32, 'wiping': 33, 'armchair': 34, 'yes': 35, 'kaleidoscope': 36, 'norwegian': 37, 'part': 38, 'proud': 39, 'loud': 40, 'enjoy': 41, 'knew': 42, 'dead': 43, 'market': 44, 'found': 45, 'fireside': 46, 'chop': 47, 'able': 48, 'tonight': 49, 'flowing': 50, 'bright': 51, 'neighbors': 52, 'paper': 53, 'vont': 54, 'anyway': 55, 'ride': 56, 'voices': 57, 'turning': 58, 'imitate': 59, 'hah': 60, 'wings': 61, "daddy's": 62, 'saying': 63, 'dance': 64, 'sunshine': 65, 'burns': 66, 'keep': 67, 'production': 68, 'certain': 69, 'anymore': 70, 'steal': 71, '

generated1000 tokens
generated2000 tokens
generated3000 tokens
generated4000 tokens
generated5000 tokens
generated6000 tokens
generated7000 tokens
training data generated


MemoryError: Unable to allocate 244. MiB for an array with shape (43548, 1467) and data type int32

In [ ]:
num_neurons = X.shape[1]
print(num_neurons)
model = keras.Sequential()
model.add(keras.layers.Dense(100, input_dim=num_neurons, activation='relu'))
model.add(keras.layers.Dense(num_neurons, activation='softmax'))

model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
history = model.fit(X, y, epochs=50, batch_size=128)
model.summary()


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])

In [ ]:
model2 = keras.Model(inputs=model.inputs, outputs=model.layers[0].output)
x_enc = model2.predict(X)
model2.summary()


In [ ]:
x_enc.shape

In [ ]:
print(x_enc)

In [ ]:
index = word_to_id.get('love')
print(index)
a = model.predict(X[index:index+1])

b = list(np.argsort(a)[0])[0:10]
print(b)


for p in range(10):
    print(id_to_word.get(b[p]))

In [ ]:
v1 = model2.predict(X[index:index+1])

idx = word_to_id.get('like')
v2 = model2.predict(X[idx:idx+1])

#print(v1)
#print(v2)
print(np.linalg.norm(v2-v1))

idk = word_to_id.get('gun')
v3 = model2.predict(X[idk:idk+1])

#print(v3)
print(np.linalg.norm(v3-v1))


In [ ]:
word_vectors = []

for i in range(num_neurons):
    word_vectors.append(one_hot_encode(i, num_neurons))

word_vectors = np.array(word_vectors)



In [ ]:
enc = model2.predict(word_vectors)
enc.shape

print(enc)

In [ ]:
p = list()
target = np.array(enc[word_to_id.get('yesterday')])
#print(target)

for i in range(num_neurons):
    p.append(np.linalg.norm(target - np.array(enc[i])))

p = np.array(p)
a = list(np.argsort(p))

lab = list()
pp = list()

fig, ax = plt.subplots()

#print(a)
for k in range(11):
    lab.append(id_to_word.get(a[k]))
    pp.append([p[a[k]]])

plt.scatter(pp[1:], np.zeros((10)))
print(lab)

pp = np.array(pp)

for k in range(1, 10):
    plt.annotate(lab[k], (pp[k], 0.003 * k))
